In [1]:
import argparse
import os
import logging
from typing import Optional

import numpy as np
import torch
from datasets import load_dataset
from evaluate import load as eval_load
from huggingface_hub import login as hf_login
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

In [3]:
def postprocess_indictrans_text(preds, labels, source_lang="eng_Latn", target_lang="mar_Deva"):
    prefix = f"{target_lang} {source_lang}"

    def remove_prefix(text: str) -> str:
        if text.startswith(prefix):
            return text[len(prefix):].strip()
        return text.strip()

    preds = [remove_prefix(p) for p in preds]
    # return labels as list-of-lists for BLEU (sacrebleu expects that)
    labels = [[remove_prefix(l)] for l in labels]
    return preds, labels

In [4]:
def _safe_extract_comet_score(comet_out: dict) -> Optional[float]:
    # Try common keys used by various COMET wrappers
    for k in ("system_score", "mean_score", "score", "value"):
        if k in comet_out:
            return float(comet_out[k])
    # Some wrappers return {"scores": [...]} or similar
    if "scores" in comet_out:
        return float(np.mean(comet_out["scores"]))
    return None

In [5]:
def compute_metrics_indictrans(
    eval_preds,
    tokenizer,
    metric_bleu,
    metric_chrf=None,
    metric_comet=None,
    metric_bleurt=None,
    source_lang="eng_Latn",
    target_lang="mar_Deva",
):
    """
    Robust compute_metrics that accepts either:
      - eval_preds as (preds, labels) tuple (older usage), or
      - eval_preds as transformers.EvalPrediction object with attributes
        .predictions, .label_ids, and (optionally) .inputs when
        include_inputs_for_metrics=True is set in training args.
    """
    # Unpack in a robust way
    if hasattr(eval_preds, "predictions"):
        preds = eval_preds.predictions
        labels = eval_preds.label_ids
        inputs = getattr(eval_preds, "inputs", None)
    else:
        # fallback if user passed a tuple (preds, labels)
        preds, labels = eval_preds
        inputs = None

    # If generation returned tuple (generated_ids, something) take first
    if isinstance(preds, tuple):
        preds = preds[0]

    # Convert tensors -> numpy arrays if necessary
    def to_numpy(x):
        if isinstance(x, torch.Tensor):
            return x.cpu().numpy()
        return x

    preds = to_numpy(preds)
    labels = to_numpy(labels)
    inputs = to_numpy(inputs) if inputs is not None else None

    # Decode predictions (list[str])
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    # Decode labels: replace -100 -> pad_token_id then decode
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    # If inputs are present (requires include_inputs_for_metrics=True), decode sources
    decoded_sources = None
    if inputs is not None:
        decoded_sources = tokenizer.batch_decode(inputs, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    # Postprocess (remove prefixes). Note: postprocess returns references as list-of-lists.
    decoded_preds, decoded_labels = postprocess_indictrans_text(decoded_preds, decoded_labels, source_lang, target_lang)

    # ----- BLEU (sacrebleu expects references as list-of-lists) -----
    try:
        bleu_out = metric_bleu.compute(predictions=decoded_preds, references=decoded_labels)
        bleu_score = float(bleu_out.get("score", 0.0))
    except Exception as e:
        logger.warning("BLEU computation failed: %s", e)
        bleu_score = 0.0

    # ----- ChrF++ -----
    chrfpp_score = None
    if metric_chrf is not None:
        try:
            chrf_out = metric_chrf.compute(predictions=decoded_preds, references=decoded_labels, word_order=2)
            chrfpp_score = float(chrf_out.get("score", np.nan))
        except Exception as e:
            logger.warning("ChrF computation failed: %s", e)

    # ----- COMET -----
    comet_score = None
    if metric_comet is not None:
        if decoded_sources is None:
            logger.warning("COMET requested but decoded_sources is None — set include_inputs_for_metrics=True in TrainingArguments")
        else:
            try:
                # COMET wants flat references (not list-of-lists)
                flat_refs = [r[0] if isinstance(r, (list, tuple)) and len(r) > 0 else r for r in decoded_labels]
                comet_out = metric_comet.compute(sources=decoded_sources, predictions=decoded_preds, references=flat_refs)
                comet_score = _safe_extract_comet_score(comet_out)
            except Exception as e:
                logger.warning("COMET computation failed: %s", e)

    # ----- BLEURT -----
    bleurt_score = None
    if metric_bleurt is not None:
        try:
            flat_refs = [r[0] if isinstance(r, (list, tuple)) and len(r) > 0 else r for r in decoded_labels]
            bleurt_out = metric_bleurt.compute(predictions=decoded_preds, references=flat_refs)
            # BLEURT returns "scores" usually
            if "scores" in bleurt_out:
                bleurt_score = float(np.mean(bleurt_out["scores"]))
            elif "score" in bleurt_out:
                bleurt_score = float(bleurt_out["score"])
        except Exception as e:
            logger.warning("BLEURT computation failed: %s", e)

    # ----- Generated length -----
    try:
        prediction_lens = [int(np.count_nonzero(p != tokenizer.pad_token_id)) for p in preds]
        gen_len = float(np.mean(prediction_lens)) if len(prediction_lens) > 0 else 0.0
    except Exception:
        gen_len = 0.0

    result = {
        "bleu": round(bleu_score, 4),
        "chrfpp": round(chrfpp_score, 4) if chrfpp_score is not None else None,
        "comet": round(comet_score, 4) if comet_score is not None else None,
        "bleurt": round(bleurt_score, 4) if bleurt_score is not None else None,
        "gen_len": round(gen_len, 4),
    }
    return result

In [6]:
def load_hf_token(path="hf_token.txt"):
    try:
        with open(path, "r") as f:
            return f.read().strip()
    except FileNotFoundError:
        raise RuntimeError(f"HF token file not found at {path}. Create it with your token.")

In [7]:
!nvidia-smi

Mon Nov 24 07:46:46 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   67C    P0             190W / 300W |  35873MiB / 81920MiB |     68%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [8]:
cuda = "1"
model_checkpoint = "ai4bharat/indictrans2-en-indic-dist-200M"
dataset = "thenlpresearcher/shalaka_iitb_en_indic_tokenized"
batch_size = 16
eval_steps = 6000
save_steps = 6000
num_train_epochs = 5
learning_rate = 1e-5
save_total_limit = 3
metric_for_best_model = "chrfpp"
greater_is_better = True
source_lang = "eng_Latn"
target_lang = "mar_Deva"
push_to_hub = True
hub_model_id=None

In [9]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
os.environ["WANDB_DISABLED"] = "true"

In [10]:
hf_token = load_hf_token()
hf_login(hf_token)

logger.info("Loading model/tokenizer from %s", model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, trust_remote_code=True)

logger.info("Loading dataset %s", dataset)
tokenized_datasets = load_dataset(dataset)

logger.info("Loading metrics")
metric_bleu = eval_load("sacrebleu")
metric_chrf = eval_load("chrf")
metric_comet = None
metric_bleurt = None
try:
    metric_comet = eval_load("comet")
except Exception as e:
    logger.warning("Could not load COMET metric: %s", e)
try:
    metric_bleurt = eval_load("bleurt")
except Exception as e:
    logger.warning("Could not load BLEURT metric: %s", e)

2025-11-24 07:47:07,254 - INFO - Loading model/tokenizer from ai4bharat/indictrans2-en-indic-dist-200M
2025-11-24 07:47:09,935 - INFO - Loading dataset thenlpresearcher/shalaka_iitb_en_indic_tokenized
2025-11-24 07:47:12,711 - INFO - Loading metrics
Fetching 5 files: 100%|████████████████████| 5/5 [00:00<00:00, 11478.66it/s]
2025-11-24 07:47:19,149 - INFO - Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
2025-11-24 07:47:29,148 - INFO - Encoder model frozen.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
2025-11-24 07:47:31,078 - WARNING - Could not load B

In [11]:
model_name = model_checkpoint.rstrip("/").split("/")[-1]
hub_model_id = f"thenlpresearcher/shalaka_fd_{model_name}_finetuned_{source_lang}_to_{target_lang}"

In [12]:
print(model_name)
print(hub_model_id)
print(push_to_hub)

indictrans2-en-indic-dist-200M
thenlpresearcher/shalaka_fd_indictrans2-en-indic-dist-200M_finetuned_eng_Latn_to_mar_Deva
True


In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"shalaka_fd_{model_name}-en-indic-iitb-finetuned-{source_lang}-to-{target_lang}",
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=eval_steps,
    save_steps=save_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=save_total_limit,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    fp16=False,
    logging_steps=100,
    seed=42,
    push_to_hub=push_to_hub,
    hub_model_id=hub_model_id,
    hub_private_repo=False,
    hub_strategy="end",
    load_best_model_at_end=True,
    metric_for_best_model=metric_for_best_model,
    greater_is_better=greater_is_better,
    include_inputs_for_metrics=True,   # IMPORTANT for COMET
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding="longest")      

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using `include_inputs_for_metrics` is deprecated and will be removed in version 5 of 🤗 Transformers. Please use `include_for_metrics` list argument instead.


In [14]:
trainer = Seq2SeqTrainer(
    model=model,                           # use local model variable
    args=training_args,                    # pass Seq2SeqTrainingArguments instance
    train_dataset=tokenized_datasets.get("train"),
    eval_dataset=tokenized_datasets.get("validation"),
    tokenizer=tokenizer,                   # important so EvalPrediction.inputs is set
    data_collator=data_collator,
    compute_metrics=lambda eval_preds: compute_metrics_indictrans(
        eval_preds,
        tokenizer=tokenizer,
        metric_bleu=metric_bleu,
        metric_chrf=metric_chrf,
        metric_comet=metric_comet,
        metric_bleurt=metric_bleurt,
        source_lang=source_lang,
        target_lang=target_lang,
    ),
)  

/tmp/ipykernel_88538/1878573675.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
2025-11-24 07:47:31,281 - WARNING - Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
logger.info("Starting fine-tuning ...")
trainer.train()

2025-11-24 07:47:33,181 - INFO - Starting fine-tuning ...


Step,Training Loss,Validation Loss


2025-11-24 08:12:11,073 - INFO - 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
2025-11-24 08:12:11,089 - INFO - GPU available: True (cuda), used: True
2025-11-24 08:12:11,090 - INFO - TPU available: False, using: 0 TPU cores
2025-11-24 08:12:11,094 - INFO - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]


In [ ]:
logger.info("Pushing final checkpoint to hub: %s", hub_model_id)
trainer.push_to_hub()

logger.info("Training finished")